In [1]:
# # Mounting GCS to colab
# # https://stackoverflow.com/questions/51715268/how-to-import-data-from-google-cloud-storage-to-google-colab

# from google.colab import auth
# auth.authenticate_user()

In [6]:
# !echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
# !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
# !apt -qq update
# !apt -qq install gcsfuse

In [11]:
# !mkdir colab_indoor
# !gcsfuse indoor-data colab_indoor
# !mkdir colab_indoor/train_3_1
# !gcsfuse indoor-data/train_3_1 colab_indoor/train_3_1
# !mkdir colab_indoor/test_3_1
# !gcsfuse indoor-data/test_3_1 colab_indoor/test_3_1

In [4]:
# !pip install torchviz

In [99]:
!ls -la -h ./colab_indoor/train_3_1

total 4.0G
-rw-r--r-- 1 root root 152M Apr  5 03:47 5a0546857ecc773753327266_train.csv
-rw-r--r-- 1 root root 144M Apr  5 03:48 5c3c44b80379370013e0fd2b_train.csv
-rw-r--r-- 1 root root 798M Apr  5 03:48 5d27075f03f801723c2e360f_train.csv
-rw-r--r-- 1 root root 219M Apr  5 03:48 5d27096c03f801723c31e5e0_train.csv
-rw-r--r-- 1 root root 129M Apr  5 03:48 5d27097f03f801723c320d97_train.csv
-rw-r--r-- 1 root root  21M Apr  5 03:48 5d27099f03f801723c32511d_train.csv
-rw-r--r-- 1 root root  25M Apr  5 03:48 5d2709a003f801723c3251bf_train.csv
-rw-r--r-- 1 root root 145M Apr  5 03:48 5d2709b303f801723c327472_train.csv
-rw-r--r-- 1 root root 206M Apr  5 03:47 5d2709bb03f801723c32852c_train.csv
-rw-r--r-- 1 root root 283M Apr  5 03:48 5d2709c303f801723c3299ee_train.csv
-rw-r--r-- 1 root root 106M Apr  5 03:48 5d2709d403f801723c32bd39_train.csv
-rw-r--r-- 1 root root  73M Apr  5 03:47 5d2709e003f801723c32d896_train.csv
-rw-r--r-- 1 root root 7.3M Apr  5 03:48 5da138274db8ce0c98bbd3d2_train.csv
-

---
## CNN Implementation
---

In [133]:
import random
from random import sample
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
from scipy.ndimage.filters import uniform_filter1d
from scipy.interpolate import interp1d
import time
from collections import defaultdict
import seaborn as sns

import scipy.stats as stats
from pathlib import Path
import glob
import pickle
from tqdm import tqdm
import random
import os
import copy
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import multiprocessing

EPOCH = 5 # default at 50
BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FOLDS = 10
# inpath = '../input/indoor-location-navigation/'
# metapath = inpath + 'metadata'
# trainpath = inpath + 'train'
# testpath = inpath + 'test'

NUM_CORES = multiprocessing.cpu_count()
print(NUM_CORES)

OUTPUT_NAME = "train_6_1_Conv1dMaxpoolDense"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

2


In [134]:
# train paths and test paths
train_files = sorted(glob.glob("./colab_indoor/train_3_1/*"))
test_files = sorted(glob.glob("./colab_indoor/test_3_1/*"))

# load submission file
sub_df = pd.read_csv("./colab_indoor/sample_submission.csv", index_col=0)
# sub_df[["site", "file", "timestamp"]] = sub_df["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
display(sub_df.head())

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,75.0,75.0


In [135]:
# Load train csv and test csv
train_df = pd.read_csv(train_files[0])
test_df = pd.read_csv(test_files[0])
display(train_df.head())
display(test_df.head())

,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,acce_x_avg,acce_y_avg,acce_z_avg,magn_x_avg,magn_y_avg,magn_z_avg,magn_st,ahrs_x_avg,ahrs_y_avg,ahrs_z_avg,gyro_x_avg,gyro_y_avg,gyro_z_avg,magn_u_x_avg,magn_u_y_avg,magn_u_z_avg,rel_diff,rel_x,rel_y,fd4804b1282054b5df1bc478fbcb73fbd1aa09cb,2253c6a0d0f7277737aa8e86e0484be805124806,2daa7fde27c0fd44e4a0133c3247be06d3c21c0b,ce0718d609ae0f06031490c462fff571210d0d0c,82385a56f25847fe6966d46bb58f302c4597953c,7f1bc20c9c21d1751f25e2ed5dbfda28e1b2554a,3fc1c8fa39c6996247670b3a9182b320dc3d9f0a,5baf7217c2d6e9596923d5e4349928f50c1a6d40,10983f664fc2be47f37d3bef4c78e2f088620fef,a35c02b945625781267821b43bd2eb609616ce65,26f99f7138f8721213e77ac21d013d4fa1716ef0,358dc9545d5faa51c88d04edf8c902068e13b106,3688b1d72f939347de1233defbb4f7ae706eb5ef,...,6f311690978e199e95ad23003eb1deb1d0b2f42b,7e44cc6c807fe154ba7fe97bca97030f6e610d2b,db2118bbf6034e55c8a594481aa87d6a67e3bfbb,d6f8e0a20a1a65b0ce10a3d58c3f3a8758a95e6f,f6b4a651fe2db1c68c3652da07f7869453adfee8,52991874f27e29074381dc2b6cf4267ba079564b,85f187bfc2e3a1eab8027c925690679c1de6b5c7,4ed61e923a531cd500928e7dcf4412d5dabdd4e6,8b59fb25bf867e2b57ba14cb71c386d202d714df,3e30d40b1f40522801cbb68a468b92414f1ccf13,98a30c53639727fd1f43c9ab4237c3c3f6153ada,aecd9cc3d0c1640e971226c67678032d65747459,1ae65ecc64fd68a95580a4d968f9c9f8cf777ae9,b71b9159ec25bd67017caa99494263086a5cd326,c26c54c0dfa013b4ee1e15a3f11bcfa8215c0a99,1b12eb877cf5261ab74957f2ca659ee89e858a42,6e53995eb99731c76e4804cb193e160d60e03cff,21d6b5544a59eac82f9bb8bf54105a41958adaf8,94f0be14f76dfbb65a8d9c920542f06e4f79fa26,a630919f315899ebf8650eb07d1097204ba5a2d3,6471b2899033a43f16bfd62b8198ef9557a49f62,43b75ab26009a503bb7cc0b4776edc6c27621ea7,27f2fa8a8809bc430be43ea21c8968ed8f98cef7,48be90ff375acd4d6a323e4d85ede7564cebd98a,380da08a7279a1947c76c0961bf2c7bfa8bb2eaf,b2ce01476d4f985ee9d0518d1eadf9ed18d236ad,9a08f3c0cf309d67555c8f9c6f64686b9ba2283d,16f0057b4591b780054a4f2867ba3fa9ec793438,2e985809d94e7cfe3dbdc78a27e17baa56bef58b,1ad3144a286d2b32af03f8e275191f8c48c77e10,8c1eceb10a2c1b8d943f836acf2444392250f685,2a1bd89115889eacd8ef93c0fa2584e238a098a8,d16001127911cd8aad3e6e09663802d8748cd68e,e95654c4ebc092ae1845ba174839a8fc0e5c92b4,35e02baaefd4e7c076667ecadce166c2d2f09977,d90ab0610e02c2adb1174feaf719b9e506564893,01ead5ee97d0329b2c228fece5a96b0358a3cab1,8c077f9959ff86dc9f8836444618c2483f12e1ba,31595a6c2bd3a944b144685a2f117715520fd417,ecb87e6c93dd3c6cdb6f6671e25e2d0d6f865875
0,1578470975068,1913,141.92305,82.461280,B1,-1,5e158ecff4c3420006d52164,5a0546857ecc773753327266,-0.648250,5.580048,8.470229,-0.424541,27.571661,-57.607026,57.607026,0.302154,0.026655,-0.049684,-0.360110,0.050747,0.082396,-7.631267,44.037420,-387.173983,1340,0.087030,0.498510,-47,-52,-54,-54,-54,-54,-59,-61,-66,-69,-76,-77,-77,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,1578470976967,3812,141.92305,82.461280,B1,-1,5e158ecff4c3420006d52164,5a0546857ecc773753327266,-0.648250,5.580048,8.470229,-0.424541,27.571661,-57.607026,57.607026,0.302154,0.026655,-0.049684,-0.360110,0.050747,0.082396,-7.631267,44.037420,-387.173983,1340,0.087030,0.498510,-67,-55,-68,-54,-54,-54,-56,-86,-66,-74,-76,-86,-86,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,1578470978867,3120,142.00758,87.245865,B1,-1,5e158ecff4c3420006d52164,5a0546857ecc773753327266,-1.326786,5.907629,7.612880,-8.777472,6.484076,-52.121773,52.121773,0.323410,0.013562,-0.137427,-0.121506,-0.051817,0.107889,-16.005221,22.998895,-381.709195,1138,0.237318,0.391589,-65,-67,-71,-71,-54,-71,-56,-86,-66,-78,-76,-85,-86,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999

,site_path_timestamp,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,acce_x_avg,acce_y_avg,acce_z_avg,magn_x_avg,magn_y_avg,magn_z_avg,magn_st,ahrs_x_avg,ahrs_y_avg,ahrs_z_avg,gyro_x_avg,gyro_y_avg,gyro_z_avg,magn_u_x_avg,magn_u_y_avg,magn_u_z_avg,rel_diff,rel_x,rel_y,eebf5db207eec2f3e041f92153d789270f346821,7805f319f3f591986effe78c5b41143180278f2d,323607d8444900d64151ee06d164738ac727bbce,b26914599f6d9ba16b43975394e1eeb9d82f4bab,02a1be3a5dab38320f879489d8a1e0f2a72768b3,5b225e187d0dec3110683a74d0c9a5a4cb2022f5,d84cce12fbfba61bf930123050f61a11e2a29310,b2546cae6e588d38618eacc557dd0385812197cf,6bc91b3951089c3a225396608b138ca178479924,8464ea586ee5479e1250f938d7c01e9bc68cefe8,b4dbb0b30caa1d0f21b7b4185ba061556cada67a,c48db7f3ed1858bb4fc191230e3d79d5eb178604,...,ead4eb320daf267710b8ec2f80b572ef14ca55f6,a19dac7dbdfd5b1628917159bb0333d0b4248b01,a5fa56c01bc636432ad0deb09ac769dd8b4bc4c5,c3403babce8901fd7a22f2752ee27e55c93137ef,e8af39b7dfe78bd5413b4af4a667d052e8cbaa77,fb91245a09b92f24ae14fc6e76e1b3549bd6a31d,0697d08b2741a320bcdcbb9624caca0e3a415356,2d199d76f05af1ff3432955b6b52163e8341dda5,b37d43c8c5f1e82765b5a403c19529d89b765d62,0076ff7a084cb2ac8c146139965ab1be296e72c4,b896a2a793aa802db868a7a0fdb29204bde99065,108716739163fd1a6e473359330704f7cd2068f7,e32d9e78dd87486b832f7723c916ca93ad686a54,1ad3144a286d2b32af03f8e275191f8c48c77e10,38f586d6a54a75a4fc9df0325de877b2bce5de6b,097599fa8b0d2bb4426ba0dae5ca9e578cdb8149,dc522df53e28d76a729dc56f81b1295a50ce4289,042e55ae2e3aae88070d507977eb97cd7468ef80,3b61536817d546393995f664b446e4ecf609ed61,295dcf2d00e2d19d02f73fc1d498ed5ca821161c,647649398bea2985feed4728d748939666dfb609,f3d9e7d629e50b8ca7e330d1d7cc11d6a75def8f,c70a6973225a173d7a0f74ca0ff93d07d140ed9d,a9f2d89eb253ffaf137bad7322e7cae41cd575d7,da8b535fe9eb7c6ea9388d1d2c4c80777da0fa2f,315dc4970761f38035d1da2d3592624abf1293ab,cb7c0d32ada93bf41f44aa2b37a5c584e894e040,96524d4078bf26b236a6ccc4efcf8bfb14189ab4,72a8a758661a41895e424613d666109e838b3d5f,af03d645e2ce58455848037f78dc79d962ec95d2,b831c303c1abd40fd8b242f902b6d900bca585af,c87c9fe93ddb4dc25d1d241c87a340ecf5be19f5,6bcdb1157c0d47e8a75d4ae864210156ebc2de96,817b99c65c4d92ab758443590680da2c3129c62b,563c56cbd1eba74106d9d4cf3adef19ddd574858,24f7fe5fa290f803c49bac656c57df514b180b4b,7e642dee5535c87ae5d65d225ee5dab01769b04c,ae2a68d930d314c1c164485b9ac5ef5beb158290,c5de94605289a9fae1869b9c73dde7806c559717,ed61d67ab40541ebb671536a7ea20125119cb964
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,2340,2331,-999.0,-999.0,-999.0,-999.0,046cfa46be49fc10834815c6,5a0546857ecc773753327266,-0.033225,4.443197,8.658677,26.840626,2.208085,-38.039814,38.039814,0.236958,0.094365,0.430658,-0.088104,-0.054522,0.037350,30.966603,10.374589,-377.064170,1135,-0.425353,0.248690,-45,-46,-46,-47,-47,-48,-48,-48,-48,-48,-49,-49,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,9508,491,-999.0,-999.0,-999.0,-999.0,046cfa46be49fc10834815c6,5a0546857ecc773753327266,-1.048206,3.798536,8.933929,23.539826,-0.836536,-28.946169,28.946169,0.173001,0.166249,0.435596,-0.125305,-0.375643,-0.616186,27.665803,7.329968,-367.970522,187,-0.537325,0.140535,-69,-47,-52,-47,-65,-47,-46,-46,-47,-54,-46,-46,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,14714,612,-999.0,-999.0,-999.0,-999.0,046cfa46be49fc10834815c6,5a0546857ecc773753327266,-1.554605,3.885587,9.000827,-4.966844,12.898208,-40.707459,40.707459,0.237424,0.024758,-0.185234,-0.062272,-0.090101,0.118882,-0.840867,21.064712,-379.731812,161,0.295503,0.550062,-69,-42,-59,-41,-58,-42,-42,-42,-41,-55,-45,-47,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,

In [136]:
# Match train and test columns
all_train_cols = list(train_df.columns)
all_test_cols = list(test_df.columns)
print("all train cols: ", len(all_train_cols), "\n", "all test cols: ", len(all_test_cols))

# get all non-overlapping columns
no_overlap_col = list(set(all_train_cols) ^ set(all_test_cols))
train_cols = [x for x in all_train_cols if x not in no_overlap_col]
test_cols = [x for x in all_test_cols if x not in no_overlap_col]
# test_cols += ["site_path_timestamp"] # test_df needs to keep "site_path_timestamp"

# filter out the df by the columns to leave
train_df = train_df[train_cols]
test_df = test_df[test_cols]

# Drop some columns not necessary as a feature
drop_cols =["wifi_ts", "floor", "file_id", "site_id"]
for df in [train_df, test_df]:
    df = df.drop(columns=drop_cols, inplace=True)

# Convert df object columns to integers and then the whole thing to tensors
for df in [train_df, test_df]:
    obj_col = list(df.select_dtypes(include=['object']).columns)
    for col in obj_col:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].values)

print(len(train_df.columns))
print(len(test_df.columns))
print(len(train_df))
print(len(test_df))
print("object dtype columns in train", train_df.select_dtypes(include=['object']).columns)
print("object dtype columns in test", test_df.select_dtypes(include=['object']).columns)

all train cols:  3424 
 all test cols:  2453
2412
2412
9296
300
object dtype columns in train Index([], dtype='object')
object dtype columns in test Index([], dtype='object')


In [137]:
# # Use when we need to consider timetamps

# # get timestamp and sort by time
# test_df[["site", "file", "timestamp"]] = test_df["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
# test_df = test_df.drop(columns=["site_path_timestamp", "site", "file"])
# test_df["timestamp"] = test_df["timestamp"].astype(int)
# # display(test_df.head())

# # sort by time
# train_df = train_df.sort_values(by=["file_id", "wifi_ts"])
# test_df = test_df.sort_values(by=["file_id", "timestamp"])
# # display(train_df.head(20))
# # display(test_df.head(20))
# # print(len(test_df.columns))

In [138]:
class IndoorDataset(Dataset):
    def __init__(self, data, flag="train"):
        self.flag = flag
        self.data = data

        all_cols = list(data.columns)
        target_cols = ["x", "y", "floor_int"]
        non_target_cols = [col for col in all_cols if col not in target_cols]
        self.features = data[non_target_cols]

        if self.flag == "train":
            self.x = data.loc[:, "x"]
            self.y = data.loc[:, "y"]
            self.f = data.loc[:, "floor_int"]

    def __len__(self):
        return len(self.features)

    def width(self):
        return self.features.shape[1]

    def __getitem__(self, index):
        features = self.features.iloc[index, :]
        features_out = torch.tensor(features.to_numpy())
        if self.flag == "train":
            x = self.x[index]
            y = self.y[index]
            f = self.f[index]
            x_out = torch.tensor(x)
            y_out = torch.tensor(y)
            f_out = torch.tensor(f)
            return features_out, x_out, y_out, f_out
        else:
            return features_out

In [139]:
# Create train and test Dataset
train_ds = IndoorDataset(train_df)
test_ds = IndoorDataset(test_df, flag="test")
one_train_ds = train_ds.__getitem__(2)
print("train ds: ", one_train_ds)
print("train ds feature len: ", len(one_train_ds[0]))
print("train ds x: ", one_train_ds[1])
print("train ds y: ", one_train_ds[2])
print("train ds f: ", one_train_ds[3])
print("test ds feature len: ", len(test_ds.__getitem__(2)))
print(train_ds.__len__())
print(test_ds.__len__())
print(train_ds.width())
print(test_ds.width())

train ds:  (tensor([ 3.1200e+03, -1.3268e+00,  5.9076e+00,  ..., -9.9900e+02,
        -9.9900e+02, -9.9900e+02], dtype=torch.float64), tensor(142.0076, dtype=torch.float64), tensor(87.2459, dtype=torch.float64), tensor(-1))
train ds feature len:  2409
train ds x:  tensor(142.0076, dtype=torch.float64)
train ds y:  tensor(87.2459, dtype=torch.float64)
train ds f:  tensor(-1)
test ds feature len:  2409
9296
300
2409
2409


In [140]:
# Create Dataloader
train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True)

In [141]:
class Conv1dMaxPoolGRU(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, gru_hidden_size):
        super(Conv1dMaxPoolGRU, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
        self.sigmoid = nn.Sigmoid()
        self.max_pool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.relu = nn.ReLU()
        self.gru = nn.GRU(input_size=out_channels, hidden_size=gru_hidden_size, num_layers=1)
        self.batch_norm = nn.BatchNorm1d(1)
        self.dropout = nn.Dropout(0.3)
        # Paper suggests flatten -> GRU -> MDN
        # But for now, just connect it to 2 Dense layers and no MDN
        self.dense_x = nn.Linear(gru_hidden_size, 1)
        self.dense_y = nn.Linear(gru_hidden_size, 1)
        self.dense_f = nn.Linear(gru_hidden_size, 1)

    def forward(self, x, prints=False):
        if prints: print("before conv: ", x.shape)
        x = self.conv(x)
        x = self.sigmoid(x)
        if prints: print("after conv + sigmoid: ", x.shape)
        x = self.max_pool(x)
        x = self.relu(x)
        if prints: print("after max_pool + relu: ", x.shape)
        x = x.transpose(1, 2)
        if prints: print("after transpose: ", x.shape)
        x = self.batch_norm(x)
        x = self.dropout(x)
        x, _ = self.gru(x)
        x = self.relu(x)
        if prints: print("after gru + relu: ", x.shape)
        # print("after gru + sigmoid: ", x[0])
        # x = x.view(x.size(0), -1)
        # print("after view: ", x.shape)
        f = self.dense_x(x)
        y = self.dense_y(x)
        x = self.dense_x(x)
        return x, y, f

In [142]:
train_length = train_ds.__len__()
train_width = train_ds.width()

model = Conv1dMaxPoolGRU(in_channels=train_width, out_channels=100, kernel_size=3, stride=2, padding=1, gru_hidden_size=200)
print(model)

# Check if it works
train_batch_sample = next(iter(train_dataloader))
print(train_batch_sample[0].shape)
print(train_batch_sample[1].shape)
# train_batch_sample_tensor = torch.cat(train_batch_sample, dim=0)
# print(train_batch_sample.shape)
train_batch_sample = train_batch_sample[0]
train_batch_sample = train_batch_sample.unsqueeze(-1)
outputs = model(train_batch_sample.float())
# print(outputs[0].shape)
# print(outputs[1].shape)
# print(outputs[2].shape)

Conv1dMaxPoolGRU(
  (conv): Conv1d(2409, 100, kernel_size=(3,), stride=(2,), padding=(1,))
  (sigmoid): Sigmoid()
  (max_pool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (gru): GRU(100, 200)
  (batch_norm): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (dense_x): Linear(in_features=200, out_features=1, bias=True)
  (dense_y): Linear(in_features=200, out_features=1, bias=True)
  (dense_f): Linear(in_features=200, out_features=1, bias=True)
)
torch.Size([64, 2409])
torch.Size([64])


In [143]:
display(train_df.head())

,wps_diff,x,y,floor_int,acce_x_avg,acce_y_avg,acce_z_avg,magn_x_avg,magn_y_avg,magn_z_avg,magn_st,ahrs_x_avg,ahrs_y_avg,ahrs_z_avg,gyro_x_avg,gyro_y_avg,gyro_z_avg,magn_u_x_avg,magn_u_y_avg,magn_u_z_avg,rel_diff,rel_x,rel_y,fd4804b1282054b5df1bc478fbcb73fbd1aa09cb,2253c6a0d0f7277737aa8e86e0484be805124806,2daa7fde27c0fd44e4a0133c3247be06d3c21c0b,ce0718d609ae0f06031490c462fff571210d0d0c,82385a56f25847fe6966d46bb58f302c4597953c,7f1bc20c9c21d1751f25e2ed5dbfda28e1b2554a,3fc1c8fa39c6996247670b3a9182b320dc3d9f0a,5baf7217c2d6e9596923d5e4349928f50c1a6d40,10983f664fc2be47f37d3bef4c78e2f088620fef,a35c02b945625781267821b43bd2eb609616ce65,26f99f7138f8721213e77ac21d013d4fa1716ef0,358dc9545d5faa51c88d04edf8c902068e13b106,3688b1d72f939347de1233defbb4f7ae706eb5ef,e62e836b877d4189e890fcc9d0d4d7d09964933a,4608951935961c98acdd8a88cb4d3dfd0358fa6c,9ad1d8c3a29b04ff542c90d2f6e05eaeddc42a97,d6a858af60bc6f39894eb9118b1aa44f8cfd6be2,...,3160e934656e963e9e0043b2f4f90e2648bf743d,8ca53a415b0219e5aa117e7e4a36d119352e4001,065fdf6b71a04ac9f116ef6c48d5ad11aa4f81d2,1e6cf70bccf9f19d88b72c8d626704099aaed9c9,6ec546bda8ada6afc76e5d5c5d6a4c7c3fce1f32,92fa127222645a92ea37393b7607e90945faf15d,215392414424d604bb113570ebd2f9da75b4761e,7a539d7ef9c0a69fd5dd0b09e98aeee85ec0194b,944b1b3f0749817e0815937b4f5efa5f719c50a2,9fa437dcd243128911d16094767209c2d04947cb,0df5ea55a86bf40b688994f1372d23ca1c7933d6,f71bfb2d30a2d98762544abce3534422d2fe99c2,a62958e9857892a6acb4d98344d93dd69a550a3e,2f81febb1050d0a35d9e2b66b724160b7fbe47a0,a43581283d31c0eae988b34ec85ca39000c680cd,c87c9fe93ddb4dc25d1d241c87a340ecf5be19f5,f9402fa4c19b3124c5a395f2bf443e0417dd0380,6ec256be9ad01a967e5eac307c10fd4d319d6b72,730d8d58fc114ba448f60535d7843c49feec7578,0f7d10d5fabcc4d22bcae8ee90e4f09d42932415,9058e469ff561b485e22afa134d49679818bf80f,b30b484a4b6aedee66258c6abe85cd4d1ecfbd9e,9c027992989747bb680bf798c91130ed4d1b8d65,67ade87897d10497a8b9e774c2cd5c3e821783db,1bb5d0affd5b295b22c204ce1b092259299b3f72,984bd7dc2a680eaaf85955f278ecbcc9cacfe36c,9537a11721507dd679188a0f165abdc43a59fae5,d479f2fe67b64cb4c00a1a547ab12dd99609824e,966c43ea0250eba77ec615eccfe18625850d65af,cda785a670b1dfb74431b92a4dff4c4eebb612e9,fb6f12c3dadd71dd9b0bb928205ad07725ab405a,adb7b69f9b8c690c9a0c8b81126f653cb6f32c72,305a7cba0b9e5c983ce179bfebfe72a92a9a66c7,e229e5b52a293e7e6f5112dd6c8f6be40789fe3f,8299cd39d69e1aec8f37ed2f895be4b9997e7a56,c26c54c0dfa013b4ee1e15a3f11bcfa8215c0a99,94f0be14f76dfbb65a8d9c920542f06e4f79fa26,a630919f315899ebf8650eb07d1097204ba5a2d3,6471b2899033a43f16bfd62b8198ef9557a49f62,1ad3144a286d2b32af03f8e275191f8c48c77e10
0,1913,141.92305,82.461280,-1,-0.648250,5.580048,8.470229,-0.424541,27.571661,-57.607026,57.607026,0.302154,0.026655,-0.049684,-0.360110,0.050747,0.082396,-7.631267,44.037420,-387.173983,1340,0.087030,0.498510,-47,-52,-54,-54,-54,-54,-59,-61,-66,-69,-76,-77,-77,-77,-77,-77,-78,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,3812,141.92305,82.461280,-1,-0.648250,5.580048,8.470229,-0.424541,27.571661,-57.607026,57.607026,0.302154,0.026655,-0.049684,-0.360110,0.050747,0.082396,-7.631267,44.037420,-387.173983,1340,0.087030,0.498510,-67,-55,-68,-54,-54,-54,-56,-86,-66,-74,-76,-86,-86,-77,-77,-77,-83,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,3120,142.00758,87.245865,-1,-1.326786,5.907629,7.612880,-8.777472,6.484076,-52.121773,52.121773,0.323410,0.013562,-0.137427,-0.121506,-0.051817,0.107889,-16.005221,22.998895,-381.709195,1138,0.237318,0.391589,-65,-67,-71,-71,-54,-71,-56,-86,-66,-78,-76,-85,-86,-77,-77,-79,-85,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-99

In [145]:
mse = nn.MSELoss()
mse = mse.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=5e-3) # default lr: 5e-3

data_dict ={}
best_loss = 1000
num_epochs = 1
best_epoch = 0
for epoch in range(EPOCH):
    print("epoch: ", epoch)
    model.train()
    losses = []
    for i, data in enumerate(train_dataloader):
        features = data[0].float()
        features = features.unsqueeze(-1)
        x = data[1].to(DEVICE).float().unsqueeze(-1)
        y = data[2].to(DEVICE).float().unsqueeze(-1)
        f = data[3].to(DEVICE).int().unsqueeze(-1)

        model = Conv1dMaxPoolGRU(in_channels=train_width, out_channels=100, kernel_size=3, stride=2, padding=1, gru_hidden_size=200)
        output = model(features, prints=False)
        output = torch.cat(output, dim=-1).squeeze(1)
        label = torch.cat([x, y, f], dim=-1)
        # print(label[0])
        # print(label.shape)
        # print(output[0])
        # print(output.shape)
        optim.zero_grad()
        loss = mse(output, label)
        loss.backward()
        optim.step()
        losses.append(loss.cpu().detach().numpy())
        print(f"loss:{np.mean(losses)} at iteration {i}")

print("Training finished")

epoch:  0
loss:7462.47900390625 at iteration 0
loss:8121.4208984375 at iteration 1
loss:7990.36279296875 at iteration 2
loss:8284.9267578125 at iteration 3
loss:7994.4208984375 at iteration 4
loss:7917.88134765625 at iteration 5
loss:8104.1650390625 at iteration 6
loss:8048.4970703125 at iteration 7
loss:7989.201171875 at iteration 8
loss:7903.95556640625 at iteration 9
loss:7859.71240234375 at iteration 10
loss:7848.92822265625 at iteration 11
loss:7759.4189453125 at iteration 12
loss:7756.8837890625 at iteration 13
loss:7720.8193359375 at iteration 14
loss:7806.58349609375 at iteration 15
loss:7868.52685546875 at iteration 16
loss:7906.7119140625 at iteration 17
loss:7882.93896484375 at iteration 18
loss:7981.95166015625 at iteration 19
loss:7976.0595703125 at iteration 20
loss:8045.9794921875 at iteration 21
loss:8017.0849609375 at iteration 22
loss:8006.689453125 at iteration 23
loss:8014.2255859375 at iteration 24
loss:8021.86376953125 at iteration 25
loss:8050.4345703125 at itera

KeyboardInterrupt: ignored

In [ ]:
# def evaluate(model, data_loader,  device='cuda'):
#     model.to(device)
#     model.eval()
#     x_list = []
#     y_list = []
#     floor_list = []
#     prexs_list = []
#     preys_list = []
#     prefloors_list = []
#     for d in tqdm(data_loader):
#         data_dict['BSSID_FEATS'] = d['BSSID_FEATS'].to(device).long()
#         data_dict['RSSI_FEATS'] = d['RSSI_FEATS'].to(device).float()
#         data_dict['site_id'] = d['site_id'].to(device).long()
#         x = d['x'].to(device).float()
#         y = d['y'].to(device).float()
#         floor = d['floor'].to(device).long()
#         x_list.append(x.cpu().detach().numpy())
#         y_list.append(y.cpu().detach().numpy())
#         floor_list.append(floor.cpu().detach().numpy())
#         xy, floor = model(data_dict)
#         prexs_list.append(xy[:, 0].cpu().detach().numpy())
#         preys_list.append(xy[:, 1].cpu().detach().numpy())
#         prefloors_list.append(floor.squeeze().cpu().detach().numpy())
#     x = np.concatenate(x_list)
#     y = np.concatenate(y_list)
#     floor = np.concatenate(floor_list)
#     prexs = np.concatenate(prexs_list)
#     preys =np.concatenate(preys_list)
#     prefloors = np.concatenate(prefloors_list)
#     eval_score = comp_metric(x, y, floor, prexs, preys, prefloors)
#     return eval_score

# def get_result(model, data_loader, device='cuda'):
#     model.eval()
#     model.to(device)
#     prexs_list = []
#     preys_list = []
#     prefloors_list = []
#     data_dict = {}
#     for d in tqdm(data_loader):
#         data_dict['BSSID_FEATS'] = d['BSSID_FEATS'].to(device).long()
#         data_dict['RSSI_FEATS'] = d['RSSI_FEATS'].to(device).float()
#         data_dict['site_id'] = d['site_id'].to(device).long()
#         xy, floor = model(data_dict)
#         prexs_list.append(xy[:, 0].cpu().detach().numpy())
#         preys_list.append(xy[:, 1].cpu().detach().numpy())
#         prefloors_list.append(floor.squeeze(-1).cpu().detach().numpy())
#     prexs = np.concatenate(prexs_list)
#     preys =np.concatenate(preys_list)
#     prefloors = np.concatenate(prefloors_list)
#     return prexs, preys, prefloors